Predicting Runs Scored: Data Wrangling

Last notebook we decided to switch our focus onto predicting runs scored
This could be useful in several capacities:

    - For predicting the over/under

    - To create features for predicting the game winner


We will focus on predicting the distribution of runs scored - that is, putting a probability on each possible value of runs scored for a team (up to some maximum value)

Predicting the distribution of a numeric target variable is known as "probabilistic regression"

In this notebook, we will "wrangle" our data such that each row represents a hitting team against a pitching team
The goal will be to predict the (distribution of the) number of runs scored by the hitting team

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
df = pd.read_csv('df_bp9.csv', low_memory=False)

Create df for runs of a single team

    - Want to "split" each game into two rows:

        - One where the home team bats against the visiting pitchers (and we try to predict the runs of the home team)

        - Other where we switch the roles of home and visiting

        - Then we can model how many runs we expect based on the hitters for that team and the opposing pitching

In [3]:
cols_ref = ['season', 'date','dblheader_code','date_dblhead','team_h','team_v',
            'league_h','league_v','over_under_line','over_under_result',
           'ballpark_id', 'day_night']

target_cols = ['runs_h', 'runs_v']

In [4]:
team_hit_stems = ['BATAVG','OBP','SLG','OBS','ERR','SB','CS']
lineup_hit_stems = ['BATAVG','OBP','SLG','OBS','SLGmod','SObat_perc']
strt_pitch_stems = ['ERA','WHIP','SO_perc','H_BB_perc','TB_BB_perc','FIP','FIP_perc']
bpen_pitch_stems = ['WHIP','SO_perc','H_BB_perc','TB_BB_perc']

team_hit_winsizes = [30,162]
lineup_hit_winsizes = [30,75,162,350]
strt_pitch_winsizes = [10,35,75]
bpen_pitch_winsizes = [10,35,75]

In [5]:
# We will make two data frames:
# version 'a': home hitting and visitor pitching
# version 'b': visitor hitting and home pitching

team_hit_features_a = [x+'_'+str(winsize)+'_h' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_a = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_h' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_a = ['Strt_'+x+'_'+str(winsize)+'_v' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_a = ['Bpen_'+x+'_'+str(winsize)+'_v' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

team_hit_features_b = [x+'_'+str(winsize)+'_v' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_b = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_v' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_b = ['Strt_'+x+'_'+str(winsize)+'_h' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_b = ['Bpen_'+x+'_'+str(winsize)+'_h' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

In [6]:
cols_a = cols_ref + ['runs_h'] + team_hit_features_a + lineup_hit_features_a + start_pitch_features_a + bpen_pitch_features_a
df_a = df.loc[:,cols_a]
df_a['home_hitting'] = 1


cols_b = cols_ref + ['runs_v'] + team_hit_features_b + lineup_hit_features_b + start_pitch_features_b + bpen_pitch_features_b
df_b = df.loc[:,cols_b]
df_b['home_hitting'] = 0

stripped_feats  = [x[:-2] for x in team_hit_features_a + lineup_hit_features_a + 
                                              start_pitch_features_a + bpen_pitch_features_a]

final_col_list = cols_ref +['runs'] + stripped_feats+ ['home_hitting']

df_a.columns = final_col_list
df_b.columns = final_col_list
df_runs = pd.concat((df_a,df_b))

In [12]:
df_runs.sample(10)

,season,date,dblheader_code,date_dblhead,team_h,team_v,league_h,league_v,over_under_line,over_under_result,ballpark_id,day_night,runs,BATAVG_30,OBP_30,SLG_30,OBS_30,ERR_30,SB_30,CS_30,BATAVG_162,OBP_162,SLG_162,OBS_162,ERR_162,SB_162,CS_162,lineup8_BATAVG_30,lineup9_BATAVG_30,lineup8_BATAVG_30_w,lineup9_BATAVG_30_w,lineup8_OBP_30,lineup9_OBP_30,lineup8_OBP_30_w,lineup9_OBP_30_w,lineup8_SLG_30,lineup9_SLG_30,lineup8_SLG_30_w,lineup9_SLG_30_w,lineup8_OBS_30,lineup9_OBS_30,lineup8_OBS_30_w,lineup9_OBS_30_w,lineup8_SLGmod_30,lineup9_SLGmod_30,lineup8_SLGmod_30_w,lineup9_SLGmod_30_w,lineup8_SObat_perc_30,lineup9_SObat_perc_30,lineup8_SObat_perc_30_w,lineup9_SObat_perc_30_w,lineup8_BATAVG_75,lineup9_BATAVG_75,lineup8_BATAVG_75_w,lineup9_BATAVG_75_w,lineup8_OBP_75,lineup9_OBP_75,lineup8_OBP_75_w,lineup9_OBP_75_w,lineup8_SLG_75,lineup9_SLG_75,lineup8_SLG_75_w,lineup9_SLG_75_w,lineup8_OBS_75,lineup9_OBS_75,lineup8_OBS_75_w,lineup9_OBS_75_w,lineup8_SLGmod_75,lineup9_SLGmod_75,lineup8_SLGmod_75_w,lineup9_SLGmod_75_w,lineup8_SObat_perc_75,lineup9_SObat_perc_75,lineup8_SObat_perc_75_w,lineup9_SObat_perc_75_w,lineup8_BATAVG_162,lineup9_BATAVG_162,lineup8_BATAVG_162_w,lineup9_BATAVG_162_w,lineup8_OBP_162,lineup9_OBP_162,lineup8_OBP_162_w,lineup9_OBP_162_w,lineup8_SLG_162,lineup9_SLG_162,lineup8_SLG_162_w,lineup9_SLG_162_w,lineup8_OBS_162,lineup9_OBS_162,lineup8_OBS_162_w,lineup9_OBS_162_w,lineup8_SLGmod_162,lineup9_SLGmod_162,lineup8_SLGmod_162_w,lineup9_SLGmod_162_w,lineup8_SObat_perc_162,lineup9_SObat_perc_162,lineup8_SObat_perc_162_w,lineup9_SObat_perc_162_w,lineup8_BATAVG_350,lineup9_BATAVG_350,lineup8_BATAVG_350_w,lineup9_BATAVG_350_w,lineup8_OBP_350,lineup9_OBP_350,lineup8_OBP_350_w,lineup9_OBP_350_w,lineup8_SLG_350,lineup9_SLG_350,lineup8_SLG_350_w,lineup9_SLG_350_w,lineup8_OBS_350,lineup9_OBS_350,lineup8_OBS_350_w,lineup9_OBS_350_w,lineup8_SLGmod_350,lineup9_SLGmod_350,lineup8_SLGmod_350_w,lineup9_SLGmod_350_w,lineup8_SObat_perc_350,lineup9_SObat_perc_350,lineup8_SObat_perc_350_w,lineup9_SObat_perc_350_w,Strt_ERA_10,Strt_WHIP_10,Strt_SO_perc_10,Strt_H_BB_perc_10,Strt_TB_BB_perc_10,Strt_FIP_10,Strt_FIP_perc_10,Strt_ERA_35,Strt_WHIP_35,Strt_SO_perc_35,Strt_H_BB_perc_35,Strt_TB_BB_perc_35,Strt_FIP_35,Strt_FIP_perc_35,Strt_ERA_75,Strt_WHIP_75,Strt_SO_perc_75,Strt_H_BB_perc_75,Strt_TB_BB_perc_75,Strt_FIP_75,Strt_FIP_perc_75,Bpen_WHIP_10,Bpen_SO_perc_10,Bpen_H_BB_perc_10,Bpen_TB_BB_perc_10,Bpen_WHIP_35,Bpen_SO_perc_35,Bpen_H_BB_perc_35,Bpen_TB_BB_perc_35,Bpen_WHIP_75,Bpen_SO_perc_75,Bpen_H_BB_perc_75,Bpen_TB_BB_perc_75,home_hitting
73378,2004,20040830,0,200408300,NYN,FLO,NL,NL,0.0,NaN,NYC17,N,6,0.273986,0.336347,0.412463,0.748810,13.0,18.0,6.0,0.262637,0.326192,0.407692,0.733885,82.0,93.0,46.0,0.278888,0.260863,0.280275,0.264406,0.346277,0.328172,0.347744,0.331797,0.433108,0.403503,0.433464,0.407586,0.779384,0.731675,0.781209,0.739384,0.365441,0.360021,0.361566,0.357211,0.132157,0.139695,0.129050,0.135932,0.270858,0.260763,0.271829,0.262922,0.338458,0.326037,0.339785,0.328813,0.439882,0.419895,0.439437,0.422032,0.778340,0.745932,0.779222,0.750846,0.387160,0.374512,0.383932,0.373205,0.141107,0.141724,0.138757,0.139524,0.270762,0.249937,0.271861,0.253575,0.331504,0.306330,0.332675,0.310586,0.430981,0.396469,0.430230,0.400176,0.762484,0.702798,0.762906,0.710762,0.365577,0.339018,0.361668,0.338862,0.135441,0.127937,0.133087,0.126764,0.272110,0.246161,0.272845,0.250122,0.334414,0.302654,0.335376,0.307557,0.433647,0.391655,0.432402,0.395865,0.768061,0.694308,0.767778,0.703422,0.369576,0.335020,0.365916,0.336105,0.136555,0.124874,0.134401,0.124414,3.453488,1.412791,0.144578,0.325301,0.425703,3.889535,0.895582,2.458399,1.243328,0.162921,0.296629,0.395506,3.282575,0.783146,3.566790,1.410332,0.170669,0.325498,0.444047,4.076015,0.940726,1.445455,0.181818,0.344156,0.487013,1.500000,0.197452,0.354565,0.473461,1.472262,0.211694,0.347782,0.462702,0
29527,1984,19840923,0,198409230,HOU,CIN,NL,NL,0.0,NaN,HOU02,N,2,0.262802,0.318750,0.378744,0.697494,30.0,21

In [13]:
df_runs.to_csv('df_runs_bp11.csv', index=False)